# Hyperparameter Tuning by Randomized-Search
In previous notebooks, we explored grid-search. This has limitations: it does not scale well when the number of parameters to tune increases.

In [4]:
import pandas as pd

adult_census = pd.read_csv("../1 Predictive Modeling Pipeline/adult.csv")

target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=[target_name, "education-num"])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, random_state=42
)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_selector = selector(dtype_include=object)
categorical_columns = categorical_selector(data)

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical_preprocessor", categorical_preprocessor, categorical_columns)
    ],
    remainder="passthrough"
)

In [7]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", HistGradientBoostingClassifier(
            random_state=42,
            max_leaf_nodes=4
        ))
    ]
)

With `GridSearchCV`, the parameters need to be specified explicitly. With randomized search, the parameter candidates are generated randomly, avoiding the regularity of the grid. *One of the dangers is that the best combination falls between the specified combinations of the grid.*

`RandomizedSearchCV` using a stochastic search. This allows us to find an optimum quicker without defining a grid.

![searchcomp.png](searchcomp.png)

We will also optimize three other parameters in additional to `learning_rate` and `max_leaf_nodes`.
- `l2_regularization`: the strength of the regularization
- `min_samples_leaf`: minimum number of samples required in a leaf
- `max_bins`: maximum number of bins to construct the histograms

In [8]:
from scipy.stats import loguniform

class loguniform_int:
    """Integer valued version of the log-uniform distribution"""

    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

In [9]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    "classifier__l2_regularization": loguniform(1e-6, 1e3),
    "classifier__learning_rate": loguniform(0.001, 10),
    "classifier__max_leaf_nodes": loguniform_int(2, 256),
    "classifier__min_samples_leaf": loguniform_int(1, 100),
    "classifier__max_bins": loguniform_int(2, 255),
}

model_random_search = RandomizedSearchCV(
    model,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    verbose=1
)
model_random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'classifier__l2_regularization': <scipy.stats....t 0x114c4e690>, 'classifier__learning_rate': <scipy.stats....t 0x114bdb460>, 'classifier__max_bins': <__main__.log...t 0x114cb5950>, 'classifier__max_leaf_nodes': <__main__.log...t 0x114c7da90>, ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will 

In [10]:
accuracy = model_random_search.score(X_test, y_test)
accuracy

0.8589796085496684

In [11]:
model_random_search.best_params_

{'classifier__l2_regularization': np.float64(9.161425176358778e-05),
 'classifier__learning_rate': np.float64(0.2804267645482436),
 'classifier__max_bins': np.int64(29),
 'classifier__max_leaf_nodes': np.int64(2),
 'classifier__min_samples_leaf': np.int64(6)}

In [12]:
column_results = [f"param_{name}" for name in param_distributions.keys()]
column_results += ["mean_test_score", "std_test_score", "rank_test_score"]

cv_results = pd.DataFrame(model_random_search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False
)


def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name


cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
2,0.000092,0.280427,2,6,29,0.849335,0.003202,1
8,0.000015,0.043570,4,2,102,0.840927,0.001467,2
3,2.937742,0.012217,136,2,8,0.833857,0.001707,3
7,0.000011,0.030993,3,25,31,0.811089,0.001192,4
6,0.000001,0.017151,3,88,185,0.799377,0.001872,5
5,1.062357,0.124055,2,61,2,0.799377,0.003214,6
9,92.664601,2.800870,137,2,6,0.789878,0.011730,7
0,125.089843,0.014612,7,15,2,0.781224,0.000608,8
4,0.000027,3.754022,34,5,5,0.724687,0.056103,9
1,0.022938,1.835532,2,12,31,0.450957,0.058925,10
